In [3]:
import yfinance as yf
import pandas as pd
import backtesting as bt
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import pandas as pd

In [5]:
def get_data(ticker:str='AAPL',from_date:str='2020-01-01',to_date:str='2022-11-30',interval:str='1d') -> pd.DataFrame:
    yf_ticker = yf.Ticker(ticker)
    return yf_ticker.history(start=from_date,end=to_date,interval=interval)

def MovingAverage(closes:pd.Series, n:int) -> pd.Series:
    return pd.Series(closes).rolling(n).mean()

In [6]:
class SmaCross(Strategy):
    sma_fast = 12
    sma_slow = 24
    
    def init(self):
        self.sma1 = self.I(MovingAverage, self.data.Close, self.sma_fast)
        self.sma2 = self.I(MovingAverage, self.data.Close, self.sma_slow)

    def next(self):
        if not self.position and crossover(self.sma1, self.sma2):
            self.buy()
        elif self.position and crossover(self.sma2, self.sma1):
            self.position.close()

In [7]:
data = get_data(ticker='AAPL',from_date='2020-01-01',to_date='2022-11-30')

In [9]:
bt = Backtest(data, SmaCross, cash=10_000, commission=0,exclusive_orders=True)

In [10]:
stats = bt.run()

In [11]:
print(stats)

Start                     2020-01-02 00:00...
End                       2022-11-29 00:00...
Duration                   1062 days 00:00:00
Exposure Time [%]                   59.536785
Equity Final [$]                 19386.314034
Equity Peak [$]                  22092.086729
Return [%]                           93.86314
Buy & Hold Return [%]               91.907399
Return (Ann.) [%]                   25.517445
Volatility (Ann.) [%]               30.254074
Sharpe Ratio                         0.843438
Sortino Ratio                        1.655825
Calmar Ratio                         0.994593
Max. Drawdown [%]                  -25.656168
Avg. Drawdown [%]                   -4.218404
Max. Drawdown Duration      456 days 00:00:00
Avg. Drawdown Duration       45 days 00:00:00
# Trades                                   15
Win Rate [%]                        53.333333
Best Trade [%]                      64.310489
Worst Trade [%]                     -9.168091
Avg. Trade [%]                    

In [12]:
print(stats._trades)

    Size  EntryBar  ExitBar  EntryPrice   ExitPrice          PnL  ReturnPct  \
0    152        69      179   65.590444  107.771979  6411.593349   0.643105   
1    144       195      211  113.233260  107.172804  -872.705639  -0.053522   
2    133       223      264  116.705132  126.593749  1315.185971   0.084732   
3    119       268      285  141.293806  128.339862 -1541.519345  -0.091681   
4    129       312      314  118.357646  121.855857   451.269171   0.029556   
5    127       317      340  123.994190  127.738572   475.536607   0.030198   
6    126       365      406  128.261720  144.518032  2048.295336   0.126743   
7    124       408      435  146.841168  144.972777  -231.680444  -0.012724   
8    122       455      515  147.108081  169.795914  2767.915630   0.154226   
9    120       532      543  172.619277  161.636031 -1317.989450  -0.063627   
10   114       563      579  170.666474  167.286261  -385.344327  -0.019806   
11   137       615      619  139.259260  131.118938 